# Initial Exploration

### Merge csvs, Preprocessing, EDA

In [5]:
import numpy as np
import pandas as pd
import glob
import os

In [31]:
'''
Merge datasets in the data folder.
Returns a dataframe of all the data.
'''
def retrieve_reviews_df():
    all_files = glob.glob(".\data\*.csv")

    df_list = []

    for filename in all_files:
        print(f"Concatenating {filename}")
        df = pd.read_csv(filename, index_col=None, header=0)
        df_list.append(df)
    
    return pd.concat(df_list, axis=0, ignore_index=True)

In [32]:
reviews_df = retrieve_reviews_df()
print(reviews_df.head())

Concatenating .\data\Reviews-1.csv
